In [1]:
%autosave 90

Autosaving every 90 seconds


Name: Jishnu Chander Ravichanderan <br>
Github Username: jishnuchander <br>
USC ID: 1144843551 <br>

In [227]:
# Importing all required libraries
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor as fl
import seaborn as sns
from scipy.stats import variation
from sklearn.impute import SimpleImputer
from sklearn import model_selection
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, hamming_loss, silhouette_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss, mean_squared_error, roc_auc_score 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, KFold
from xgboost import  XGBClassifier
from imblearn.over_sampling import SMOTE
from scipy.spatial.distance import hamming
import warnings
warnings.filterwarnings('ignore')

(b) Each instance has three labels: Families, Genus, and Species. Each of the labels
has multiple classes. We wish to solve a multi-class and multi-label problem.
One of the most important approaches to multi-label classification is to train a
classifier for each label (binary relevance). We first try this approach:

i. Research exact match and hamming score/ loss methods for evaluating multi-
label classification and use them in evaluating the classifiers in this problem.

In [5]:
path = os.getcwd()+"\Homework_7_Data\\Anuran_Calls\\Frogs_MFCCs.csv"

In [6]:
df = pd.read_csv(path)
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [14]:
df.columns

Index(['MFCCs_ 1', 'MFCCs_ 2', 'MFCCs_ 3', 'MFCCs_ 4', 'MFCCs_ 5', 'MFCCs_ 6',
       'MFCCs_ 7', 'MFCCs_ 8', 'MFCCs_ 9', 'MFCCs_10', 'MFCCs_11', 'MFCCs_12',
       'MFCCs_13', 'MFCCs_14', 'MFCCs_15', 'MFCCs_16', 'MFCCs_17', 'MFCCs_18',
       'MFCCs_19', 'MFCCs_20', 'MFCCs_21', 'MFCCs_22', 'Family', 'Genus',
       'Species', 'RecordID'],
      dtype='object')

In [18]:
df['Family'].value_counts()

Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: Family, dtype: int64

In [22]:
df_labels = df.copy()

In [24]:
le1 = LabelEncoder()
le1.fit(df_labels.Family)
df_labels['Family'] = le1.transform(df_labels.Family)

le2 = LabelEncoder()
le2.fit(df_labels.Genus)
df_labels['Genus'] = le2.transform(df_labels.Genus)

le3 = LabelEncoder()
le3.fit(df_labels.Species)
df_labels['Species'] = le3.transform(df_labels.Species)

In [25]:
df_labels['Family'].value_counts()

3    4420
2    2165
1     542
0      68
Name: Family, dtype: int64

In [28]:
df_labels.drop('RecordID', axis=1, inplace=True)

In [31]:
train_df, test_df = train_test_split(df_labels, test_size=0.3)

In [32]:
print(train_df.shape, test_df.shape)

(5036, 25) (2159, 25)


In [35]:
train_df.columns

Index(['MFCCs_ 1', 'MFCCs_ 2', 'MFCCs_ 3', 'MFCCs_ 4', 'MFCCs_ 5', 'MFCCs_ 6',
       'MFCCs_ 7', 'MFCCs_ 8', 'MFCCs_ 9', 'MFCCs_10', 'MFCCs_11', 'MFCCs_12',
       'MFCCs_13', 'MFCCs_14', 'MFCCs_15', 'MFCCs_16', 'MFCCs_17', 'MFCCs_18',
       'MFCCs_19', 'MFCCs_20', 'MFCCs_21', 'MFCCs_22', 'Family', 'Genus',
       'Species'],
      dtype='object')

In [38]:
label_columns = ['Family', 'Genus', 'Species']
X_train = train_df.drop(label_columns, axis=1)
X_test = test_df.drop(label_columns, axis=1)

In [39]:
Y_train_1 = train_df['Family']
Y_train_2 = train_df['Genus']
Y_train_3 = train_df['Species']
Y_test_1 = test_df['Family']
Y_test_2 = test_df['Genus']
Y_test_3 = test_df['Species']

In [68]:
Y_test = test_df['Family'].tolist()
Y_test.extend(test_df['Genus'].tolist())
Y_test.extend(test_df['Species'].tolist())

ii. Train a SVM for each of the labels, using Gaussian kernels and one versus
all classifiers. Determine the weight of the SVM penalty and the width of
the Gaussian Kernel using 10 fold cross validation.1 You are welcome to try
to solve the problem with both standardized 2 and raw attributes and report
the results.

In [97]:
# Determining optimal values with raw attributes

c_vals = [0.1, 1, 10, 100]
gamma_vals = [0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3]

best_score = 0
svm_hs = []

for c in c_vals:
    for gamma in gamma_vals:
#         smoter = SMOTE(random_state=42)
#         print('alpha', alpha)

        print(c, gamma)
        kf = KFold(n_splits=10)
        avg_hs = []

        for train_index,  test_index in kf.split(X_train):

            x_train_svm, test_x = X_train.iloc[train_index.tolist(),:], X_train.iloc[test_index.tolist(),:]
            y_train_1_svm, test_y_1 = np.array(Y_train_1)[train_index.tolist()], np.array(Y_train_1)[test_index.tolist()]
            y_train_2_svm, test_y_2 = np.array(Y_train_2)[train_index.tolist()], np.array(Y_train_2)[test_index.tolist()]
            y_train_3_svm, test_y_3 = np.array(Y_train_3)[train_index.tolist()], np.array(Y_train_3)[test_index.tolist()]

            Y_test = test_y_1.tolist()
            Y_test.extend(test_y_2.tolist())
            Y_test.extend(test_y_3.tolist())
            
            clf1 = OneVsRestClassifier(SVC(C=c, gamma=gamma))
            clf1.fit(x_train_svm, y_train_1_svm)
            y_pred_1_svm = clf1.predict(test_x)
#             print(hamming_loss(test_y_1, y_pred_1_svm))

            clf2 = OneVsRestClassifier(SVC(C=c, gamma=gamma))
            clf2.fit(x_train_svm, y_train_2_svm)
            y_pred_2_svm = clf2.predict(test_x)
#             print(hamming_loss(test_y_2, y_pred_2_svm))
            
            clf3 = OneVsRestClassifier(SVC(C=c, gamma=gamma))
            clf3.fit(x_train_svm, y_train_3_svm)
            y_pred_3_svm = clf3.predict(test_x)
#             print(hamming_loss(test_y_3, y_pred_3_svm))

            Y_pred = y_pred_1_svm.tolist()
            Y_pred.extend(y_pred_2_svm)
            Y_pred.extend(y_pred_3_svm)
#               X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(x_train_xg, y_train_xg)

#             x = XGBClassifier(random_state=42,reg_alpha=alpha, objective='binary:logistic', verbosity=0, n_jobs=-1, eval_metric='logloss')
#             xgb.fit(X_train_fold_upsample,y_train_fold_upsample)

#             pred = xgb.predict(test_x)
#             f1 = f1_score(test_y, pred)
        
            avg_hs.append(1- hamming_loss(Y_test, Y_pred))

        svm_hs.append(np.mean(avg_hs))
        print(np.mean(avg_hs))
        if np.mean(avg_hs) > best_score:
            best_score = np.mean(avg_hs)
            best_c = c
            best_gamma = gamma

    print(best_c, best_gamma)

0.1 0.7
0.9314863464714358
0.1 0.9
0.9392315419651404
0.1 1.1
0.9441968559016273
0.1 1.3
0.949029894705839
0.1 1.5
0.9534651877097206
0.1 1.7
0.9582305111132149
0.1 1.9
0.9619390798068732
0.1 2.1
0.9638591730043023
0.1 2.3
0.9659112783615764
0.1 2.3
1 0.7
0.9754435030031454
1 0.9
0.9812014158435629
1 1.1
0.9852394889919742
1 1.3
0.9872906739457434
1 1.5
0.988283526355097
1 1.7
0.9890774401211777
1 1.9
0.9895406660565707
1 2.1
0.9900701610443161
1 2.3
0.9904011118474338
1 2.3
10 0.7
0.9905986041423418
10 0.9
0.9907970168407545
10 1.1
0.9909959554840272
10 1.3
0.9911947626410846
10 1.5
0.9916581200626927
10 1.7
0.9919897282968855
10 1.9
0.9922544100476506
10 2.1
0.992320679100003
10 2.3
0.9928504370601787
10 2.3
100 0.7
0.9915258449304176
100 0.9
0.9917245206012602
100 1.1
0.9917911841122578
100 1.3
0.9923210735586482
100 1.5
0.9926526817928408
100 1.7
0.9926526817928408
100 1.9
0.9925200122019209
100 2.1
0.9927845624664711
100 2.3
0.9929172320573911
100 2.3


In [98]:
print('The best C and gamma values are: ', best_c, best_gamma)

The best C and gamma values are:  100 2.3


In [99]:
# Standardizing the attrubutes

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [105]:
X_train_scaled = pd.DataFrame(X_train_scaled)

In [108]:
#Determining the optimal values with standardized attributes

c_vals = [0.001, 0.01, 0.1, 1]
gamma_vals = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5]

best_score = 0
svm_hs = []

for c in c_vals:
    for gamma in gamma_vals:
#         smoter = SMOTE(random_state=42)
#         print('alpha', alpha)

        print(c, gamma)
        kf = KFold(n_splits=10)
        avg_hs = []

        for train_index,  test_index in kf.split(X_train_scaled):

            x_train_svm, test_x = X_train_scaled.iloc[train_index.tolist(),:], X_train_scaled.iloc[test_index.tolist(),:]
            y_train_1_svm, test_y_1 = np.array(Y_train_1)[train_index.tolist()], np.array(Y_train_1)[test_index.tolist()]
            y_train_2_svm, test_y_2 = np.array(Y_train_2)[train_index.tolist()], np.array(Y_train_2)[test_index.tolist()]
            y_train_3_svm, test_y_3 = np.array(Y_train_3)[train_index.tolist()], np.array(Y_train_3)[test_index.tolist()]

            Y_test = test_y_1.tolist()
            Y_test.extend(test_y_2.tolist())
            Y_test.extend(test_y_3.tolist())
            
            clf1 = OneVsRestClassifier(SVC(C=c, gamma=gamma))
            clf1.fit(x_train_svm, y_train_1_svm)
            y_pred_1_svm = clf1.predict(test_x)
#             print(hamming_loss(test_y_1, y_pred_1_svm))

            clf2 = OneVsRestClassifier(SVC(C=c, gamma=gamma))
            clf2.fit(x_train_svm, y_train_2_svm)
            y_pred_2_svm = clf2.predict(test_x)
#             print(hamming_loss(test_y_2, y_pred_2_svm))
            
            clf3 = OneVsRestClassifier(SVC(C=c, gamma=gamma))
            clf3.fit(x_train_svm, y_train_3_svm)
            y_pred_3_svm = clf3.predict(test_x)
#             print(hamming_loss(test_y_3, y_pred_3_svm))

            Y_pred = y_pred_1_svm.tolist()
            Y_pred.extend(y_pred_2_svm)
            Y_pred.extend(y_pred_3_svm)
#               X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(x_train_xg, y_train_xg)

#             x = XGBClassifier(random_state=42,reg_alpha=alpha, objective='binary:logistic', verbosity=0, n_jobs=-1, eval_metric='logloss')
#             xgb.fit(X_train_fold_upsample,y_train_fold_upsample)

#             pred = xgb.predict(test_x)
#             f1 = f1_score(test_y, pred)
        
            avg_hs.append(1- hamming_loss(Y_test, Y_pred))

        svm_hs.append(np.mean(avg_hs))
        print(np.mean(avg_hs))
        if np.mean(avg_hs) > best_score:
            best_score = np.mean(avg_hs)
            best_c = c
            best_gamma = gamma

    print(best_c, best_gamma)

0.001 0.1
0.6033160823419272
0.001 0.3
0.5966985126279362
0.001 0.5
0.5937198239136608
0.001 0.7
0.5916675870701716
0.001 0.9
0.5912042296485637
0.001 1.1
0.5912051500520684
0.001 1.3
0.5909404683013033
0.001 1.5
0.5901460285903626
0.001 0.1
0.01 0.1
0.8874063818149305
0.01 0.3
0.7476797942503708
0.01 0.5
0.6796347575920141
0.01 0.7
0.6488567273607035
0.01 0.9
0.6162902216331639
0.01 1.1
0.596565448578371
0.01 1.3
0.5910727434335785
0.01 1.5
0.5901460285903626
0.01 0.1
0.1 0.1
0.9723339855049596
0.1 0.3
0.9511535285640654
0.1 0.5
0.8944933573164189
0.1 0.7
0.8266480482186248
0.1 0.9
0.7627084056507515
0.1 1.1
0.7176964667024309
0.1 1.3
0.6896991069456279
0.1 1.5
0.673416117054288
0.1 0.1
1 0.1
0.9880191075767616
1 0.3
0.9789524756224557
1 0.5
0.9605515057801342
1 0.7
0.942215227155585
1 0.9
0.9025700295581013
1 1.1
0.8679532592802971
1 1.3
0.8382998306457552
1 1.5
0.813808813783963
1 0.1


In [109]:
print('The best C and gamma values are: ', best_c, best_gamma)

The best C and gamma values are:  1 0.1


iii. Repeat 1(b)ii with L1-penalized SVMs.3 Remember to standardize the attributes. Determine the weight of the SVM penalty using 10 fold cross validation.

In [116]:
c_vals = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

best_score = 0
svm_hs = []

for c in c_vals:
    print(c)
    kf = KFold(n_splits=10)
    avg_hs = []

    for train_index,  test_index in kf.split(X_train_scaled):

        x_train_svm, test_x = X_train_scaled.iloc[train_index.tolist(),:], X_train_scaled.iloc[test_index.tolist(),:]
        y_train_1_svm, test_y_1 = np.array(Y_train_1)[train_index.tolist()], np.array(Y_train_1)[test_index.tolist()]
        y_train_2_svm, test_y_2 = np.array(Y_train_2)[train_index.tolist()], np.array(Y_train_2)[test_index.tolist()]
        y_train_3_svm, test_y_3 = np.array(Y_train_3)[train_index.tolist()], np.array(Y_train_3)[test_index.tolist()]

        Y_test = test_y_1.tolist()
        Y_test.extend(test_y_2.tolist())
        Y_test.extend(test_y_3.tolist())
        
        clf1 = LinearSVC(penalty='l1', C=c, dual=False)
        clf1.fit(x_train_svm, y_train_1_svm)
        y_pred_1_svm = clf1.predict(test_x)
#             print(hamming_loss(test_y_1, y_pred_1_svm))

        clf2 = LinearSVC(penalty='l1', C=c, dual=False)
        clf2.fit(x_train_svm, y_train_2_svm)
        y_pred_2_svm = clf2.predict(test_x)
#             print(hamming_loss(test_y_2, y_pred_2_svm))

        clf3 = LinearSVC(C=c, penalty='l1', dual=False)
        clf3.fit(x_train_svm, y_train_3_svm)
        y_pred_3_svm = clf3.predict(test_x)
#             print(hamming_loss(test_y_3, y_pred_3_svm))

        Y_pred = y_pred_1_svm.tolist()
        Y_pred.extend(y_pred_2_svm)
        Y_pred.extend(y_pred_3_svm)
#               X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(x_train_xg, y_train_xg)

#             x = XGBClassifier(random_state=42,reg_alpha=alpha, objective='binary:logistic', verbosity=0, n_jobs=-1, eval_metric='logloss')
#             xgb.fit(X_train_fold_upsample,y_train_fold_upsample)

#             pred = xgb.predict(test_x)
#             f1 = f1_score(test_y, pred)

        avg_hs.append(1- hamming_loss(Y_test, Y_pred))

    svm_hs.append(np.mean(avg_hs))
    print(np.mean(avg_hs))
    if np.mean(avg_hs) > best_score:
        best_score = np.mean(avg_hs)
        best_c = c

    print(best_c)

0.001
0.8196337845940231
0.001
0.01
0.9161309918268168
0.01
0.1
0.9414862675797069
0.1
1
0.9473771129834748
1
10
0.9487681056518035
10
100
0.9488343747041561
100
1000
0.9489005122702936
1000


In [117]:
print('The best C value is : ', best_c)

The best C value is :  1000


iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy
class imbalance. Report your conclusions about the classifiers you trained.

In [121]:
c_vals = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

best_score = 0
svm_hs = []

for c in c_vals:
    smoter = SMOTE(random_state=42)
    print(c)
    kf = KFold(n_splits=10)
    avg_hs = []

    for train_index,  test_index in kf.split(X_train_scaled):

        x_train_svm, test_x = X_train_scaled.iloc[train_index.tolist(),:], X_train_scaled.iloc[test_index.tolist(),:]
        y_train_1_svm, test_y_1 = np.array(Y_train_1)[train_index.tolist()], np.array(Y_train_1)[test_index.tolist()]
        y_train_2_svm, test_y_2 = np.array(Y_train_2)[train_index.tolist()], np.array(Y_train_2)[test_index.tolist()]
        y_train_3_svm, test_y_3 = np.array(Y_train_3)[train_index.tolist()], np.array(Y_train_3)[test_index.tolist()]

        X_train_upsample_1, y_train_upsample_1 = smoter.fit_resample(x_train_svm, y_train_1_svm)
        X_train_upsample_2, y_train_upsample_2 = smoter.fit_resample(x_train_svm, y_train_2_svm)
        X_train_upsample_3, y_train_upsample_3 = smoter.fit_resample(x_train_svm, y_train_3_svm)
        
        Y_test = test_y_1.tolist()
        Y_test.extend(test_y_2.tolist())
        Y_test.extend(test_y_3.tolist())
        
        clf1 = LinearSVC(C=c, penalty='l1', dual=False)
        clf1.fit(X_train_upsample_1, y_train_upsample_1)
        y_pred_1_svm = clf1.predict(test_x)
#             print(hamming_loss(test_y_1, y_pred_1_svm))

        clf2 = LinearSVC(C=c, penalty='l1', dual=False)
        clf2.fit(X_train_upsample_2, y_train_upsample_2)
        y_pred_2_svm = clf2.predict(test_x)
#             print(hamming_loss(test_y_2, y_pred_2_svm))

        clf3 = LinearSVC(C=c, penalty='l1', dual=False)
        clf3.fit(X_train_upsample_3, y_train_upsample_3)
        y_pred_3_svm = clf3.predict(test_x)
#             print(hamming_loss(test_y_3, y_pred_3_svm))

        Y_pred = y_pred_1_svm.tolist()
        Y_pred.extend(y_pred_2_svm)
        Y_pred.extend(y_pred_3_svm)
#               X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(x_train_xg, y_train_xg)

#             x = XGBClassifier(random_state=42,reg_alpha=alpha, objective='binary:logistic', verbosity=0, n_jobs=-1, eval_metric='logloss')
#             xgb.fit(X_train_fold_upsample,y_train_fold_upsample)

#             pred = xgb.predict(test_x)
#             f1 = f1_score(test_y, pred)

        avg_hs.append(1- hamming_loss(Y_test, Y_pred))

    svm_hs.append(np.mean(avg_hs))
    print(np.mean(avg_hs))
    if np.mean(avg_hs) > best_score:
        best_score = np.mean(avg_hs)
        best_c = c

    print(best_c)

0.001
0.8784082541786319
0.001
0.01
0.9241463914923159
0.01
0.1
0.9345365636866632
0.1
1
0.9351324592129762
1
10
0.9355298105546617
10
100
0.9354635415023089
10
1000
0.9353972724499565
10


In [122]:
print('The best C value is : ', best_c)

The best C value is :  10


#### Conclusions:
- Standardizing the features decreased the best penalty parameter value and the gamma parameter by orders of magnitude.
- Using SMOTE to upsample and accomodate for class imbalance will result in a better model than the other classifiers.
- Linear SVM has the highest C value since small C values give  a large number of missclassified examples as the data is not linearly separable.

Q2: Monte-Carlo Simulation: Perform the following procedures 50 times, and report
the average and standard deviation of the 50 Hamming Distances that you calculate.

(a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split
the data into train and test, as we are not performing supervised learning in this
exercise). Choose k in {1; 2; ... ; 50} automatically based on one of the methods
provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any
other method you know.

(b) In each cluster, determine which family is the majority by reading the true labels.
Repeat for genus and species.

(c) Now for each cluster you have a majority label triplet (family, genus, species).
Calculate the average Hamming distance, Hamming score, and Hamming loss5
between the true labels and the labels assigned by clusters.

In [130]:
X = df_labels.drop(label_columns, axis=1)
Y = df_labels[label_columns]

In [237]:
loss = []
score = []
distance = []

for m in range(50):
    # We use silhouette score to find the best k
    best_score = -1
    for k in range(2, 51):
        km = KMeans(n_clusters=k, random_state=m)
        km.fit(X)
        s_score = silhouette_score(X, km.labels_)
#         print(k, s_score, m)
        if s_score > best_score:
            best_score = s_score
            best_k = k
    print('The best k value is: ', best_k)
    final_km = KMeans(n_clusters=best_k, random_state=m)
    final_km.fit(X)
    cluster_labels = final_km.labels_
    df_labels['cluster_label'] = cluster_labels
    family_labels = df_labels.groupby('cluster_label')['Family'].value_counts().unstack().idxmax(axis=1).tolist()
    genus_labels = df_labels.groupby('cluster_label')['Genus'].value_counts().unstack().idxmax(axis=1).tolist()
    species_labels = df_labels.groupby('cluster_label')['Species'].value_counts().unstack().idxmax(axis=1).tolist()
    
    family_cluster = []
    genus_cluster = []
    species_cluster = []

    for i in range(len(cluster_labels)):
    #     print(i)
        family_cluster.append(family_labels[cluster_labels[i]])
        genus_cluster.append(genus_labels[cluster_labels[i]])
        species_cluster.append(species_labels[cluster_labels[i]])

    final_labels = family_cluster
    final_labels.extend(genus_cluster)
    final_labels.extend(species_cluster)
    
    Y_true = df_labels['Family'].tolist()
    Y_true.extend(df_labels['Genus'].tolist())
    Y_true.extend(df_labels['Species'].tolist())
    
    l = hamming_loss(Y_true, final_labels)
    s = 1 - l
    d = hamming(Y_true, final_labels) * len(Y_true)
    loss.append(l)
    score.append(s)
    distance.append(d)
    print('Iteration: ', m)
    print('hamming loss: ', l)
    print('hamming score: ', s)
    print('hamming distance: ', d)

The best k value is:  4
Iteration:  0
hamming loss:  0.22177438035672922
hamming score:  0.7782256196432707
hamming distance:  4787.0
The best k value is:  4
Iteration:  1
hamming loss:  0.2224229789205467
hamming score:  0.7775770210794533
hamming distance:  4801.0
The best k value is:  4
Iteration:  2
hamming loss:  0.24526291406069028
hamming score:  0.7547370859393097
hamming distance:  5294.0
The best k value is:  4
Iteration:  3
hamming loss:  0.2224229789205467
hamming score:  0.7775770210794533
hamming distance:  4801.0
The best k value is:  4
Iteration:  4
hamming loss:  0.2224229789205467
hamming score:  0.7775770210794533
hamming distance:  4801.0
The best k value is:  4
Iteration:  5
hamming loss:  0.2224229789205467
hamming score:  0.7775770210794533
hamming distance:  4801.0
The best k value is:  4
Iteration:  6
hamming loss:  0.2224229789205467
hamming score:  0.7775770210794533
hamming distance:  4801.0
The best k value is:  4
Iteration:  7
hamming loss:  0.222422978920

In [238]:
len(distance)

50

In [239]:
print('Average hamming distance, score and loss: ', np.average(distance), np.average(score), np.average(loss))

Average hamming distance, score and loss:  4829.06 0.7762770442436876 0.22372295575631226


In [240]:
print('Standard deviation of hamming distance: ', np.std(distance))

Standard deviation of hamming distance:  223.18632664211304


ISLR 12.6.2

![HW7.jpeg](HW7.jpeg)

(c) Cluster (1,2) and Cluster (3, 4)

(d) Cluster ((1,2),3) and Cluster (4)

![HW7_2.jpeg](HW7_2.jpeg)